In [1]:
#imports
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF 
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [2]:
#start SPark Content
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
sc = SparkSession.builder.appName("yelp").getOrCreate()

In [4]:
#"review_id","user_id","business_id","stars","date","text","useful","funny","cool"

#1. Clean the dataset

df = spark.read.csv( "Yelp_review.csv", header=True)

In [80]:
#business_id: string (nullable = true)
# |-- weekday: string (nullable = true)
# |-- hour: string (nullable = true)
# |-- checkins: string (nullable = true)
df1 =spark.read.csv("yelp_checkin_1.csv", header=True)

In [6]:
df.printSchema()
df.show()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|  null| null|null|
|Staff was very he...|                   0|                   0|    0|      null|                null|  null| null|null|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6

In [8]:

""" |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- categories: string (nullable = true)"""
df2 =spark.read.csv("yelp_business.csv", header=True)

In [9]:

df2.printSchema()
df2.show()

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- categories: string (nullable = true)

+--------------------+--------------------+------------------+--------------------+--------------+-------------+-----------+-------------+--------------+------------+------------+-------+--------------------+
|         business_id|                name|      neighborhood|             address|          city|        state|postal_code|     latitude|     longitude|       stars|review_count|is_open|          categories|
+--------------------+--------------------+------------

In [10]:
df1.printSchema()
df1.show()

root
 |-- business_id: string (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- checkins: string (nullable = true)

+--------------------+-------+-----+--------+
|         business_id|weekday| hour|checkins|
+--------------------+-------+-----+--------+
|3Mc-LxcqeguOXOVT_...|    Tue| 0:00|      12|
|SVFx6_epO22bZTZnK...|    Wed| 0:00|       4|
|vW9aLivd4-IorAfSt...|    Tue|14:00|       1|
|tEzxhauTQddACyqdJ...|    Fri|19:00|       1|
|CEyZU32P-vtMhgqRC...|    Tue|17:00|       1|
|9dn5pee_n2dWQfN57...|    Sun| 3:00|       5|
|6Zk5F7fsTr8n2CJTl...|    Wed| 1:00|       4|
|OE_IDW5w_W97sBcZv...|    Sat| 1:00|       1|
|gy5pr5bFAjOL5rERS...|    Sat|15:00|       1|
|r2-eAhGANXlcgQy89...|    Mon|19:00|       1|
|wiBxwslJAGDdZ2nmL...|    Mon|16:00|       1|
|08jURVR_eDvXq9scd...|    Sat|23:00|       1|
|BlWPjh2WuvQSlmYmg...|    Sun| 0:00|       1|
|6DO_2jivyxcL6SXGP...|    Wed|23:00|       1|
|wrJROwvVKHvdMVIZO...|    Thu|21:00|       3|
|uU8up3h

In [11]:
df

DataFrame[review_id: string, user_id: string, business_id: string, stars: string, date: string, text: string, useful: string, funny: string, cool: string]

In [12]:
#df = df.withColumnRenamed("stars", "label")
df = df.withColumn("label", df["stars"].cast("double"))
#df = df.where(col("label").isNotNull())
df = df.dropna(subset=['label', 'text', 'funny', 'cool',"useful"])
df.show()
df.printSchema()

+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+-----+
|           review_id|             user_id|         business_id|stars|      date|                text|              useful|               funny|                cool|label|
+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+-----+
|HRPm3vEZ_F-33TYVT...|_4iMDXbXZ1p1ONG29...|8QWPlVQ6D-OExqXoa...|    5|2014-09-24|Cycle Pub Las Veg...|                   1|                   0|                   0|  5.0|
|Ia-w-nR1FrlzsiuEi...|u0LXt3Uea_GidxRW1...|Eox_Qq74oaFZ-Yjth...|    3|2011-07-18|Service is really...|                   1|                   1|                   1|  3.0|
|hkyyWaX-EMiIkvyu1...|nsOf58RZjMTn8V94E...|djyIZW8gVNWby8wau...|    5|2017-08-23|Suzanne was able ...|                   0|                 

In [13]:
df = df.select('text', 'label')
df.show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|Cycle Pub Las Veg...|  5.0|
|Service is really...|  3.0|
|Suzanne was able ...|  5.0|
|We always go to t...|  1.0|
|This place is alw...|  1.0|
|Horrible customer...|  1.0|
|I bought a groupo...|  1.0|
|This place is alw...|  4.0|
|Sometimes it feel...|  4.0|
|I have since swit...|  3.0|
|I have eaten at s...|  2.0|
|"Okay, so from wh...|  2.0|
|"While I really e...|  2.0|
|"Came here with m...|  4.0|
|"Our friends got ...|  4.0|
|A little on the p...|  4.0|
|"Went here with m...|  4.0|
|This is a newer l...|  3.0|
|Been here twice n...|  4.0|
|Went in on a whim...|  5.0|
+--------------------+-----+
only showing top 20 rows



In [14]:

df = df.filter(df.label.isin(2.0,4.0))
df.show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|This place is alw...|  4.0|
|Sometimes it feel...|  4.0|
|I have eaten at s...|  2.0|
|"Okay, so from wh...|  2.0|
|"While I really e...|  2.0|
|"Came here with m...|  4.0|
|"Our friends got ...|  4.0|
|A little on the p...|  4.0|
|"Went here with m...|  4.0|
|Been here twice n...|  4.0|
|We stopped in her...|  4.0|
|I often stop here...|  4.0|
|Not much here oth...|  4.0|
|I hiked this cany...|  4.0|
|Holy portion size...|  4.0|
|Flavor was actual...|  4.0|
|I don't do ice cr...|  4.0|
|I ate here with a...|  4.0|
|As a vegetarian, ...|  2.0|
|"The only downsid...|  4.0|
+--------------------+-----+
only showing top 20 rows



In [15]:
df = df.limit(1000)
df.show()

+--------------------+-----+
|                text|label|
+--------------------+-----+
|This place is alw...|  4.0|
|Sometimes it feel...|  4.0|
|I have eaten at s...|  2.0|
|"Okay, so from wh...|  2.0|
|"While I really e...|  2.0|
|"Came here with m...|  4.0|
|"Our friends got ...|  4.0|
|A little on the p...|  4.0|
|"Went here with m...|  4.0|
|Been here twice n...|  4.0|
|We stopped in her...|  4.0|
|I often stop here...|  4.0|
|Not much here oth...|  4.0|
|I hiked this cany...|  4.0|
|Holy portion size...|  4.0|
|Flavor was actual...|  4.0|
|I don't do ice cr...|  4.0|
|I ate here with a...|  4.0|
|As a vegetarian, ...|  2.0|
|"The only downsid...|  4.0|
+--------------------+-----+
only showing top 20 rows



In [16]:
#2 ML pipeline
tokenizer = Tokenizer(inputCol = "text", outputCol = "words") 
TF = HashingTF(inputCol = tokenizer.getOutputCol(), outputCol = "tfFeatures")
idf = IDF(inputCol = "tfFeatures",outputCol = "features")
lr = LogisticRegression(maxIter = 10, regParam = 0.001)

In [17]:
pipeline = Pipeline(stages = [tokenizer,TF,idf,lr])
model = pipeline.fit(df)

In [18]:

#3. Test data set, unlabeled (id, text)
test = spark.createDataFrame([
     (4, "Worst restaurant ever"),
     (5, "This restaurant is absolutely the best"),
     (6, "best hotel ever"),
     (7, "I love this hotel")
     ],["id","text"])

In [19]:

#4. Make Prediction
prediction = model.transform(test)

selected = prediction.select("id","text","probability","prediction")
for row in selected.collect():
    id, text, prob, pred = row
    print("(%d, %s) --> prediction = %f probability =%s" %(id, text, pred, prob))

(4, Worst restaurant ever) --> prediction = 2.000000 probability =[0.0011268009173979953,0.0011268009173979953,0.49915003742199504,0.0011268009173979953,0.4974695598258109]
(5, This restaurant is absolutely the best) --> prediction = 4.000000 probability =[0.0006235246950701271,0.0006235246950701271,0.08555191769563698,0.0006235246950701271,0.9125775082191526]
(6, best hotel ever) --> prediction = 4.000000 probability =[0.0008863673087221466,0.0008863673087221466,0.1819838549889905,0.0008863673087221466,0.815357043084843]
(7, I love this hotel) --> prediction = 4.000000 probability =[0.0007918249925373259,0.0007918249925373259,0.13696054669456556,0.0007918249925373259,0.8606639783278225]


In [20]:
prediction

DataFrame[id: bigint, text: string, words: array<string>, tfFeatures: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [23]:
from sklearn import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


In [78]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# UDF for converting column type from vector to double type
unlist = udf(lambda x: list(x)[0])

# Iterating over columns to be scaled
for i in ["checkins"]:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df1 = pipeline.fit(df1).transform(df1).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")
    print(i)

print("After Scaling :")

checkins
After Scaling :


In [81]:
#df1= df1["checkins"].cast("double")
df1 = df1.withColumn("checkins", df1["checkins"].cast("double"))

In [82]:
df1.show()

+--------------------+-------+-----+--------+
|         business_id|weekday| hour|checkins|
+--------------------+-------+-----+--------+
|3Mc-LxcqeguOXOVT_...|    Tue| 0:00|    12.0|
|SVFx6_epO22bZTZnK...|    Wed| 0:00|     4.0|
|vW9aLivd4-IorAfSt...|    Tue|14:00|     1.0|
|tEzxhauTQddACyqdJ...|    Fri|19:00|     1.0|
|CEyZU32P-vtMhgqRC...|    Tue|17:00|     1.0|
|9dn5pee_n2dWQfN57...|    Sun| 3:00|     5.0|
|6Zk5F7fsTr8n2CJTl...|    Wed| 1:00|     4.0|
|OE_IDW5w_W97sBcZv...|    Sat| 1:00|     1.0|
|gy5pr5bFAjOL5rERS...|    Sat|15:00|     1.0|
|r2-eAhGANXlcgQy89...|    Mon|19:00|     1.0|
|wiBxwslJAGDdZ2nmL...|    Mon|16:00|     1.0|
|08jURVR_eDvXq9scd...|    Sat|23:00|     1.0|
|BlWPjh2WuvQSlmYmg...|    Sun| 0:00|     1.0|
|6DO_2jivyxcL6SXGP...|    Wed|23:00|     1.0|
|wrJROwvVKHvdMVIZO...|    Thu|21:00|     3.0|
|uU8up3hGwW9qnzQD1...|    Fri| 0:00|     1.0|
|JHPq9KvMuJaC7CPAT...|    Sun|16:00|     1.0|
|vyzfT3_KzLi8WmInw...|    Thu| 9:00|     1.0|
|E4T0rQBJa0e4HIliB...|    Thu|23:0

In [39]:
import pandas as pd


In [40]:
print(pd.to_numeric(df1.checkins, errors='coerce'))

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [46]:
print (pd.to_numeric(df1.checkins, errors='coerce').notnull())
#0     True
#1    False
#2     True
#Name: b, dtype: bool

df1 = df1[pd.to_numeric(df1.checkins, errors='coerce').notnull()]
print (df1)


ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [45]:
df1=df1.dropna()